In [2]:
# Load the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import time
import pickle
import random
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the merged dataset (you can replace this with the path to your file)
merged_data_df = pd.read_pickle('final_data.pkl')  # Or use read_csv if needed

merged_data_df.shape

(44441, 2064)

In [3]:
merged_data_df['articleType'].unique()

array(['Shirts', 'Jeans', 'Watches', 'Track Pants', 'Tshirts', 'Socks',
       'Casual Shoes', 'Belts', 'Flip Flops', 'Handbags', 'Tops', 'Bra',
       'Sandals', 'Shoe Accessories', 'Sweatshirts', 'Deodorant',
       'Formal Shoes', 'Bracelet', 'Lipstick', 'Flats', 'Kurtas',
       'Waistcoat', 'Sports Shoes', 'Shorts', 'Briefs', 'Sarees',
       'Perfume and Body Mist', 'Heels', 'Sunglasses', 'Innerwear Vests',
       'Pendant', 'Nail Polish', 'Laptop Bag', 'Scarves', 'Rain Jacket',
       'Dresses', 'Night suits', 'Skirts', 'Wallets', 'Blazers', 'Ring',
       'Kurta Sets', 'Clutches', 'Shrug', 'Backpacks', 'Caps', 'Trousers',
       'Earrings', 'Camisoles', 'Boxers', 'Jewellery Set', 'Dupatta',
       'Capris', 'Lip Gloss', 'Bath Robe', 'Mufflers', 'Tunics',
       'Jackets', 'Trunk', 'Lounge Pants', 'Face Wash and Cleanser',
       'Necklace and Chains', 'Duffel Bag', 'Sports Sandals',
       'Foundation and Primer', 'Sweaters', 'Free Gifts', 'Trolley Bag',
       'Tracksuits', 'S

In [4]:
import pandas as pd

# Assuming df is your DataFrame with 44,444 rows already loaded

# Define the label mappings based on your previous categorization
label_mappings = {
    'Shirts': 'Tops',
    'Tshirts': 'Tops',
    'Sweatshirts': 'Tops',
    'Tunics': 'Tops',
    'Tops': 'Tops',
    'Jeans': 'Bottoms',
    'Trousers': 'Bottoms',
    'Shorts': 'Bottoms',
    'Jeggings': 'Bottoms',
    'Capris': 'Bottoms',
    'Lounge Pants': 'Bottoms',
    'Track Pants': 'Bottoms',
    'Casual Shoes': 'Footwear',
    'Sandals': 'Footwear',
    'Flip Flops': 'Footwear',
    'Flats': 'Footwear',
    'Heels': 'Footwear',
    'Sports Shoes': 'Footwear',
    'Booties': 'Footwear',
    'Watches': 'Accessories',
    'Belts': 'Accessories',
    'Handbags': 'Accessories',
    'Backpacks': 'Accessories',
    'Caps': 'Accessories',
    'Sunglasses': 'Accessories',
    'Scarves': 'Accessories',
    'Jewellery Set': 'Accessories',
    'Bracelet': 'Accessories',
    'Ring': 'Accessories',
    'Necklace and Chains': 'Accessories',
    'Earrings': 'Accessories',
    'Pendant': 'Accessories',
    'Bra': 'Innerwear',
    'Innerwear Vests': 'Innerwear',
    'Night suits': 'Sleepwear',
    'Nightdress': 'Sleepwear',
    'Baby Dolls': 'Sleepwear',
    'Churidar': 'Ethnic Wear',
    'Salwar and Dupatta': 'Ethnic Wear',
    'Salwar': 'Ethnic Wear',
    'Kurtas': 'Ethnic Wear',
    'Kurtis': 'Ethnic Wear',
    'Lehenga Choli': 'Ethnic Wear',
    'Patiala': 'Ethnic Wear',
    'Sarees': 'Ethnic Wear',
    'Jackets': 'Seasonal Clothing',
    'Rain Jacket': 'Seasonal Clothing',
    'Lounge Tshirts': 'Seasonal Clothing',
    'Ties': 'Seasonal Clothing',
    'Ties and Cufflinks': 'Seasonal Clothing',
    'Swimwear': 'Seasonal Clothing',
    'Deodorant': 'Personal Care',
    'Lipstick': 'Personal Care',
    'Foundation and Primer': 'Personal Care',
    'Sunscreen': 'Personal Care',
    'Body Lotion': 'Personal Care',
    'Face Wash and Cleanser': 'Personal Care',
    'Makeup Remover': 'Personal Care',
    'Face Serum and Gel': 'Personal Care'
}

# Function to map articleType to the corresponding label
def apply_label(article):
    return label_mappings.get(article, 'Other')  # Default to 'Other' if not found

# Apply the function to create a new column for labels
merged_data_df['Label'] = merged_data_df['articleType'].apply(apply_label)


# Display the first few rows to verify
print(merged_data_df.head())


        id  articleType  gender__Unisex  gender__Women  season_Spring  \
0  15970.0       Shirts             0.0            0.0            0.0   
1  39386.0        Jeans             0.0            0.0            0.0   
2  59263.0      Watches             0.0            1.0            0.0   
3  21379.0  Track Pants             0.0            0.0            0.0   
4  53759.0      Tshirts             0.0            0.0            0.0   

   season_Summer  season_Winter  season_na  usage__Ethnic  usage__Formal  ...  \
0            0.0            0.0        0.0            0.0            0.0  ...   
1            1.0            0.0        0.0            0.0            0.0  ...   
2            0.0            1.0        0.0            0.0            0.0  ...   
3            0.0            0.0        0.0            0.0            0.0  ...   
4            1.0            0.0        0.0            0.0            0.0  ...   

   feature_2039  feature_2040  feature_2041  feature_2042  feature_2043  \

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import time

# Track total time for process
total_start_time = time.time()

# Create and save candidate pairs based on attributes
def generate_candidate_pairs(df):
    start_time = time.time()
    tops = df[df['articleType'] == 'Tops']
    bottoms = df[df['articleType'].isin(['Jeans', 'Trousers', 'Shorts'])]

    candidate_pairs = []
    for _, top_row in tops.iterrows():
        for _, bottom_row in bottoms.iterrows():
            if is_matching_attributes(top_row, bottom_row):  # Define attribute matching logic here
                candidate_pairs.append((top_row['id'], bottom_row['id']))

    candidate_pairs_df = pd.DataFrame(candidate_pairs, columns=['Top_ID', 'Bottom_ID'])
    candidate_pairs_df.to_pickle("candidate_pairs.pkl")
    
    end_time = time.time()
    print(f"Total candidate pairs generated: {len(candidate_pairs_df)}")
    print(f"Time taken to generate candidate pairs: {end_time - start_time:.2f} seconds")



In [6]:

# Check if attributes match
def is_matching_attributes(top_row, bottom_row):
    # Define your matching logic here for attributes like gender, season, usage
    matching_gender = (top_row['gender__Unisex'] == bottom_row['gender__Unisex'] or 
                       top_row['gender__Women'] == bottom_row['gender__Women'])
    matching_season = (top_row['season_Spring'] == bottom_row['season_Spring'] or
                       top_row['season_Summer'] == bottom_row['season_Summer'] or
                       top_row['season_Winter'] == bottom_row['season_Winter'] or
                       top_row['season_na'] == bottom_row['season_na'])
    matching_usage = (top_row['usage__Ethnic'] == bottom_row['usage__Ethnic'] or
                      top_row['usage__Formal'] == bottom_row['usage__Formal'] or
                      top_row['usage__Other'] == bottom_row['usage__Other'] or
                      top_row['usage__Sports'] == bottom_row['usage__Sports'])
    return matching_gender and matching_season and matching_usage



In [8]:

# Track total time for process
total_start_time = time.time()
# Run the pairing and matching process
#generate_candidate_pairs(merged_data_df)
#process_matching_pairs(merged_data_df)

total_end_time = time.time()
print(f"Total time taken for the entire process: {total_end_time - total_start_time:.2f} seconds")

Total time taken for the entire process: 0.00 seconds


In [7]:

# Process candidate pairs with cosine similarity and save matches

def process_matching_pairs(df, batch_size=500):
    candidate_pairs_df = pd.read_pickle("candidate_pairs.pkl")
    num_batches = int(np.ceil(len(candidate_pairs_df) / batch_size))
    
    similarity_start_time = time.time()
    
    for batch_num in range(num_batches):
        batch_start_time = time.time()
        batch_pairs = candidate_pairs_df.iloc[batch_num * batch_size : (batch_num + 1) * batch_size]
        matching_results = []

        for _, (top_id, bottom_id) in batch_pairs.iterrows():
            top_row = df[df['id'] == top_id].iloc[0]
            bottom_row = df[df['id'] == bottom_id].iloc[0]

            # Calculate cosine similarity for features
            top_features = top_row[[f'feature_{i}' for i in range(2048)]].values
            bottom_features = bottom_row[[f'feature_{i}' for i in range(2048)]].values
            image_similarity = cosine_similarity([top_features], [bottom_features])[0][0]

            # Define a threshold for cosine similarity to determine a match
            if image_similarity >= 0.8:
                matching_results.append({'Top_ID': top_id, 'Bottom_ID': bottom_id, 'Is_Matching': True})

        # Save each batch of matches
        with open(f'matching_results_batch_{batch_num + 1}.pkl', 'wb') as f:
            pickle.dump(matching_results, f)

        batch_end_time = time.time()
        print(f"Batch {batch_num + 1} of {num_batches} completed in {batch_end_time - batch_start_time:.2f} seconds and saved.")

    similarity_end_time = time.time()
    print(f"Total time taken for cosine similarity calculations: {similarity_end_time - similarity_start_time:.2f} seconds")


In [9]:
candidate_data_df = pd.read_pickle('candidate_pairs.pkl')  # Or use read_csv if needed


candidate_data_df.shape

(2961927, 2)

In [10]:
print(candidate_data_df.describe(include='all'))

             Top_ID     Bottom_ID
count  2.961927e+06  2.961927e+06
mean   2.874044e+04  2.730874e+04
std    1.628784e+04  1.506340e+04
min    1.978000e+03  1.567000e+03
25%    1.276900e+04  1.335100e+04
50%    3.095600e+04  2.700600e+04
75%    4.177800e+04  3.980700e+04
max    5.989800e+04  5.981400e+04


In [11]:
candidate_data_df.head()

,Top_ID,Bottom_ID
0,42419.0,39386.0
1,42419.0,18005.0
2,42419.0,54924.0
3,42419.0,26994.0
4,42419.0,10257.0


In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming merged_data_df and candidate_data_df are already defined
similarity_threshold = 0.7  # Adjust as needed
results = []

# Define the number of pairs you want to generate
target_pairs_per_label = 25000  # Total target pairs will be 50,000

# Get feature vectors in a numpy array
features = merged_data_df[[f'feature_{i}' for i in range(2048)]].values

# Use counters to track the number of pairs generated
positive_count = 0
negative_count = 0

# Unique IDs
top_ids = candidate_data_df['Top_ID'].unique()
bottom_ids = candidate_data_df['Bottom_ID'].unique()

# Generate pairs until target is reached
np.random.seed(42)  # For reproducibility
while positive_count < target_pairs_per_label or negative_count < target_pairs_per_label:
    random_top_id = np.random.choice(top_ids)
    random_bottom_id = np.random.choice(bottom_ids)

    # Get the feature vectors for the randomly selected IDs
    top_features = features[merged_data_df['id'] == random_top_id]
    bottom_features = features[merged_data_df['id'] == random_bottom_id]

    if top_features.size > 0 and bottom_features.size > 0:
        similarity = cosine_similarity(top_features, bottom_features)[0][0]
        
        # Classify based on similarity threshold
        if positive_count < target_pairs_per_label and similarity >= similarity_threshold:
            label = 1
            positive_count += 1
        elif negative_count < target_pairs_per_label and similarity < similarity_threshold:
            label = 0
            negative_count += 1
        else:
            continue  # Skip this iteration if no labels can be assigned
        
        # Store the pair with its label
        results.append((random_top_id, random_bottom_id, label))
        
        # Print progress after every 1000 pairs
        if (positive_count + negative_count) % 1000 == 0:
            print(f"Pairs generated: Positive: {positive_count}, Negative: {negative_count}")

# Convert the results to a DataFrame
siamese_pairs_df = pd.DataFrame(results, columns=['Top_ID', 'Bottom_ID', 'Label'])

# Save the DataFrame to a pickle file
siamese_pairs_df.to_pickle("siamese_pairs3.pkl")

# Print final summary
print(f"Total pairs generated: {len(siamese_pairs_df)}")


Pairs generated: Positive: 96, Negative: 904
Pairs generated: Positive: 200, Negative: 1800
Pairs generated: Positive: 307, Negative: 2693
Pairs generated: Positive: 422, Negative: 3578
Pairs generated: Positive: 514, Negative: 4486
Pairs generated: Positive: 612, Negative: 5388
Pairs generated: Positive: 712, Negative: 6288
Pairs generated: Positive: 820, Negative: 7180
Pairs generated: Positive: 917, Negative: 8083
Pairs generated: Positive: 1005, Negative: 8995
Pairs generated: Positive: 1110, Negative: 9890
Pairs generated: Positive: 1208, Negative: 10792
Pairs generated: Positive: 1325, Negative: 11675
Pairs generated: Positive: 1441, Negative: 12559
Pairs generated: Positive: 1539, Negative: 13461
Pairs generated: Positive: 1662, Negative: 14338
Pairs generated: Positive: 1768, Negative: 15232
Pairs generated: Positive: 1867, Negative: 16133
Pairs generated: Positive: 1989, Negative: 17011
Pairs generated: Positive: 2103, Negative: 17897
Pairs generated: Positive: 2224, Negative:

In [12]:
siamese_pairs_df = pd.read_pickle('siamese_pairs3.pkl')  # Or use read_csv if needed

# Display the first few rows to verify the loaded data
siamese_pairs_df.shape

(50000, 3)

In [13]:
label_distribution = siamese_pairs_df['Label'].value_counts()
print(f"Label Distribution:\n{label_distribution}")


Label Distribution:
Label
0    25000
1    25000
Name: count, dtype: int64


In [14]:
# Extract features and labels
top_ids = siamese_pairs_df['Top_ID'].values
bottom_ids = siamese_pairs_df['Bottom_ID'].values
labels = siamese_pairs_df['Label'].values

In [19]:
# Define the Siamese model
def create_siamese_model(input_shape):
    input = Input(shape=input_shape)

    # Shared layers
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    
    # Create the model
    model = Model(inputs=input, outputs=x)
    return model

In [20]:
# Create the base network
input_shape = (2048,)  # Change this if your features have a different shape
base_network = create_siamese_model(input_shape)


In [21]:
# Define inputs for the two branches
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

In [22]:
# Get the encoded features for both inputs
encoded_a = base_network(input_a)
encoded_b = base_network(input_b)

In [23]:
# Calculate the absolute difference
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K


In [24]:
# Define a custom distance function
def euclidean_distance(vects):
    return K.sqrt(K.sum(K.square(vects[0] - vects[1]), axis=1, keepdims=True))

# Calculate distance
distance = Lambda(euclidean_distance)([encoded_a, encoded_b])

In [25]:
# Create the model
siamese_model = Model(inputs=[input_a, input_b], outputs=distance)



In [26]:
# Compile the model
siamese_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [27]:
# Print the model summary
siamese_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 2048)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_2 (InputLayer)    │ (None, 2048)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional (Functional)       │ (None, 64)                │         270,528 │ input_layer_1[0][0],       │
│                               │                           │                 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda (Lambda)               │ (None, 1)                 │               0 │ functional[0][0],          │
│                               │                           │                 │ functional[1][0]           │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 270,528 (1.03 MB)

 Trainable params: 270,528 (1.03 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
from keras.optimizers import Adam

# Change the learning rate
learning_rate = 0.001  # Try a smaller learning rate
adam = Adam(learning_rate=learning_rate)

# Compile the model
siamese_model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])


In [30]:
# Create training pairs and labels
pairs = np.array(list(zip(top_ids, bottom_ids)))  # Pair up top and bottom features
labels = labels  # Labels corresponding to these pairs (already extracted)

# Split into training and validation sets (e.g., 80% train, 20% val)
from sklearn.model_selection import train_test_split

pairs_train, pairs_val, y_train, y_val = train_test_split(pairs, labels, test_size=0.2, random_state=42)

# Prepare input data for training
X_train_a = np.array([pair[0] for pair in pairs_train])  # First elements of pairs
X_train_b = np.array([pair[1] for pair in pairs_train])  # Second elements of pairs

# Prepare input data for validation
X_val_a = np.array([pair[0] for pair in pairs_val])  # First elements of validation pairs
X_val_b = np.array([pair[1] for pair in pairs_val])  # Second elements of validation pairs

class_weight ={0: 1., 1: 5.}

In [ ]:
# Train the model
history = siamese_model.fit(
    [X_train_a, X_train_b],  # Input for the model
    y_train,                 # Labels
    validation_data=([X_val_a, X_val_b], y_val),  # Validation data
    epochs=10,               # Set the number of epochs
    batch_size=32,            # Choose a batch size
    class_weight=class_weight
)


In [92]:
# Evaluate the model on validation data
val_loss, val_accuracy = siamese_model.evaluate([X_val_a, X_val_b], y_val)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Generate predictions for further analysis (e.g., classification report, confusion matrix)
y_pred = siamese_model.predict([X_val_a, X_val_b])
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Classification Report
print(classification_report(y_val, y_pred_binary))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred_binary)
print("Confusion Matrix:\n", cm)

# ROC AUC Score
roc_auc = roc_auc_score(y_val, y_pred)
print("ROC AUC Score:", roc_auc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4983 - loss: 1.2764
Validation Loss: 1.2597678899765015
Validation Accuracy: 0.5055999755859375
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
              precision    recall  f1-score   support

           0       0.51      1.00      0.67      5056
           1       0.00      0.00      0.00      4944

    accuracy                           0.51     10000
   macro avg       0.25      0.50      0.34     10000
weighted avg       0.26      0.51      0.34     10000

Confusion Matrix:
 [[5056    0]
 [4944    0]]
ROC AUC Score: 0.24397864468118882


C:\Users\amitk\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amitk\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amitk\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
